# <center>**ML School by REU Data Scinece Club**</center>

----
## <center> Неделя №7 </center>

На этой неделе вам предстоит написать Adaboost, основанный на Decision tree. Это послдений алгоритм, который мы будем писать сами

Импортируем нужные библиотеки

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

Загрузим данные, мы будем пытаться предсказать космические тела - галактики и звезды(колонка class). Что обозначает каждая колонка нам не особо важно также как и класс. Данные уже обработаны. Если вам интересно почитать о них, то вот [ссылка](https://www.sdss.org/dr18/) на официальный сайт

In [ ]:
!git clone https://github.com/MeinRespect/REU-Ml-SCHOOL.git

Cloning into 'REU-Ml-SCHOOL'...
remote: Enumerating objects: 16, done.
remote: Counting objects: 100% (16/16), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 16 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (16/16), 7.49 MiB | 2.77 MiB/s, done.


In [ ]:
df = pd.read_csv('/content/REU-Ml-SCHOOL/galaxy_data.csv').drop('objid', axis=1)
df.sample(10)

,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
53812,192.594818,41.653645,19.29268,19.01945,18.73147,18.74532,18.64649,3893,301,2,213,1639321861496010000,QSO,0.996737,1456,53115,42
84457,333.880589,19.528483,18.95940,18.01330,17.68846,17.52180,17.47788,8108,301,4,78,8532187258501220000,STAR,-0.000764,7578,56956,428
75703,245.471966,20.455140,18.99878,17.73205,17.78983,17.88865,17.86207,4010,301,3,219,2086294511352310000,STAR,-0.000422,1853,53566,7
1951,239.259702,19.802401,19.20889,18.27878,17.97433,17.84095,17.78939,4671,301,2,74,4426173207813580000,STAR,-0.000197,3931,55350,948
71735,149.688235,13.317682,17.07770,15.91564,15.40394,15.17165,15.07710,5972,301,1,227,3656974931222180000,STAR,-0.000049,3248,54880,189
41953,122.850913,43.510115,19.13361,17.58177,16.68143,16.19486,15.83331,1889,301,6,133,615945622112987000,GALAXY,0.137891,547,51959,285
77567,206.420076,40.835684,18.62784,17.72205,17.45501,17.38285,17.37100,3893,301,4,282,5302051323533090000,STAR,0.000014,4709,55720,686
57543,147.980274,5.711963,17.31614,15.96821,15.23004,14.80249,14.49460,2986,301,3,196,1119186884317180000,GALAXY,0.073265,994,52725,154
63543,249.054527,28.507019,19.09457,17.89545,17.35317,17.01033,16.85222,3964,301,2,254,3319296762904080000,GALAXY,0.143062,2948,54553,523
54944,204.388497,44.858293,19.02803,18.93016,18.64405,18.61915,18.68755,3716,301,1,129,1649525878603930000,QSO,1.166212,1465,53082,300


В наших данных три класса. Сейчас мы не будем решать задачу мультиклассовой классификации, поэтому просто удалим 1 класс и изменим обозначения на -1 и 1 

In [ ]:
df['class'].value_counts()

GALAXY    51323
STAR      38096
QSO       10581
Name: class, dtype: int64

In [ ]:
df = df[df['class'].isin(['GALAXY', 'STAR'])]
df['class'].replace({'GALAXY':-1, 'STAR':1}, inplace=True)
df

,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,47.372545,0.820621,18.69254,17.13867,16.55555,16.34662,16.17639,4849,301,5,771,8168632633242440000,1,0.000115,7255,56597,832
1,116.303083,42.455980,18.47633,17.30546,17.24116,17.32780,17.37114,6573,301,6,220,9333948945297330000,1,-0.000093,8290,57364,868
2,172.756623,-8.785698,16.47714,15.31072,15.55971,15.72207,15.82471,5973,301,1,13,3221211255238850000,1,0.000165,2861,54583,42
3,201.224207,28.771290,18.63561,16.88346,16.09825,15.70987,15.43491,4649,301,3,121,2254061292459420000,-1,0.058155,2002,53471,35
4,212.817222,26.625225,18.88325,17.87948,17.47037,17.17441,17.05235,4649,301,3,191,2390305906828010000,-1,0.072210,2123,53793,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99994,332.714103,25.583064,19.53736,17.41563,16.41278,15.95257,15.58614,8116,301,3,67,6708188165264660000,-1,0.076411,5958,56213,278
99996,173.757382,36.441603,18.33687,17.30365,17.16037,17.14895,17.14419,4504,301,2,111,2265404129658560000,1,-0.000497,2012,53493,340
99997,205.426531,38.499053,17.50690,15.63152,15.22328,15.04469,15.28668,4382,301,4,97,2257446413900210000,-1,0.004587,2005,53472,62
99998,337.135144,-9.635967,19.33946,17.21436,16.29697,15.86745,15.51556,2576,301,2,105,811847537492257000,-1,0.084538,721,52228,268


Разделите данные на train и test указав random_state = 1, test_size=0.25

In [ ]:
# Ваш код тут

# **Задание 1. Функции для оптмизации (4 балла)**

Реализуем несколько полезных функций. Начнем с функции подсчета ошибки дерева - error_rate

$$error\ rate = {\frac{количество\ ошибочно\ предсказанных\ значений}{общее\ количество\ значений}} $$

In [ ]:
def error_rate(y_true, y_pred):
    
    # Ваш код тут
    
    return error_rate

Теперь давайте реализуем подсчет alpha, функция будет определять вес конкретного дерева во всей модели.

$$alpha = 0.5 *{\log_e ({\frac{1 - error\ rate}{error\ rate}})}$$

In [ ]:
def alpha(error_rate):

    # Ваш код тут
    
    return alpha

# **Задание 2. Adaboost (12 баллов)**

Теперь мы можем написать наш Adaboost. В нем разрешается использовать DescionTreeClassifier(для них параметры можете выбрать сами). В sklearn можно взать веса для модели и настраивать их, главное не забудьте про их нормализацию, что бы они не были слишком большими

In [ ]:
class Adaboost():
    def __init__(self, n_estimators=50, learning_rate=1.0, base_classifier=DecisionTreeClassifier()):

        self.n_estimators = n_estimators # количество моделей в adaboost
        self.learning_rate = learning_rate # скорость обучение
        self.base_classifier = base_classifier # модель которую мы используем

        self.alpha = [] # тут будут храниться значения alpha для каждой модели
        self.classifiers = [] # тут будут храниться сами модели
    
    def train(self, X, y):
        
       # Задайте начальные веса

       # Теперь вам нужно сделать цикл, который проходиться по указаному количеству моделей.
       # Модели и alpha добавляйте в self.alpha и self.classifiers
       # При обучении каждой модели в метод fit вам нужно передавать sample_weight=weights

       # Весь алгоритм отлично описан в лекции
    
    def predict(self, X):
        
        n_samples = len(X)
        y_pred = np.zeros(n_samples)

        for alpha, classifier in zip(self.alpha, self.classifiers):
            y_pred += alpha * classifier.predict(X)

        return np.sign(y_pred)

Протестируйте как работает ваш алгоритм и оцените резальтат по метрике roc_auc. Она должна быть выше 0.9

In [ ]:
# Ваш код тут

# **Задание 3. Catboost (4 балла)**

Теперь мы можем попровать настоящего монстра среди классического ML - Catboost от яндекса. Его не нужно настраивать, хотя и можно(у него тоже есть GridSearch но не из Sklearn). Оцените его резульат по roc_auc. 

In [ ]:
!pip install catboost

In [ ]:
# Ваш код тут